<a href="https://colab.research.google.com/github/arashHarirpoosh/FaceRecognition/blob/main/RetinaFaceTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import lib's

In [ ]:
!pip install pytube
!pip install retinaface
!pip install deepface
!pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 5.9 MB 7.2 MB/s 
     |████████████████████████████████| 421.8 MB 19 kB/s 
     |████████████████████████████████| 49 kB 7.7 MB/s 
     |████████████████████████████████| 3.8 MB 55.3 MB/s 
     |████████████████████████████████| 50 kB 8.6 MB/s 
     |████████████████████████████████| 448 kB 69.6 MB/s 
     |████████████████████████████████| 11.5 MB 10.4 MB/s 
     |███████████▊                    | 18.6 MB 32.1 MB/s eta 0:00:02ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/

In [ ]:
# !rm ./*jpg

In [ ]:
from pytube import YouTube
import dlib
from retinaface import RetinaFace
from deepface import DeepFace
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cv2
import os
import threading
import time
import torch

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#Codes

In [ ]:
def check_tracker_quality(face_trackers, base_image, quality_thresh_hold=7):
    # Update all the trackers and remove the ones for which the update
    # indicated the quality was not good enough
    fids_to_delete = []
    for fid in face_trackers.keys():
        trackingQuality = face_trackers[fid].update(base_image)

        # If the tracking quality is good enough, we must delete
        # this tracker
        if trackingQuality < quality_thresh_hold:
            fids_to_delete.append(fid)

    for fid in fids_to_delete:
        print("Removing tracker " + str(fid) + " from list of trackers")
        del face_trackers[fid]
    return face_trackers


def create_new_tracker(current_face_id, frame, loc):
    print("Creating new tracker " + str(current_face_id))

    x, y, w, h = loc
    # Create and store the tracker
    tracker = dlib.correlation_tracker()
    tracker.start_track(frame,
                        dlib.rectangle(x - int(w / 4),
                                       y - int(h / 4),
                                       x + w + int(w / 4),
                                       y + h + int(h / 4)))
    return tracker

In [ ]:
def do_recognize_person(face_names, fid, face_img):
    verified_identity = None
    try:
        face_img_bgr = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        df = DeepFace.find(face_img_bgr,
                           db_path="/content/FaceDataBase",
                           enforce_detection=False,
                           detector_backend='retinaface',
                           model_name='ArcFace',
                          #  model=model,
                           distance_metric="euclidean_l2")
        print(df)
        print()
        # filtered_faced = df[df['ArcFace_euclidean_l2'] <= 1]
        closest_identity = df.loc[df['ArcFace_euclidean_l2'].idxmin()]
        print(closest_identity['identity'])
        # verified_identity = closest_identity['identity'].split('/')[-2].split('\\')[1]
        verified_identity = closest_identity['identity'].split('/')[-2]

        print()
        print(verified_identity)
        del df
    except (ValueError, cv2.error):
        print('No Face Detected')
        # all_faces = RetinaFace.detect_faces(data)
        # print(all_faces)
        # cv2.imshow('face', face_img)
        # cv2.waitKey()

    num_of_zeros = 3 - len(str(fid))
    if verified_identity is None:
        face_names[fid] = {
            'name': '{person_id}'.format(person_id=num_of_zeros * '0' + str(fid)),
            'detected': False
        }
    else:
        face_names[fid] = {
            'name': verified_identity,
            'dir': '{person_id}'.format(person_id=verified_identity + '/' + num_of_zeros * '0' + str(fid)),
            'detected': True
        }


def check_existence_of_new_faces(result_list, face_trackers,
                                 face_names, frame, current_face_id):
    verified_trackers = []
    for result in result_list:
        x, y, x1, y1 = result['facial_area']
        w, h = abs(x1 - x), abs(y1 - y)

        # Calculate the center point
        x_bar = (x + x1) / 2
        y_bar = (y + y1) / 2

        # Variable holding information which faceid we
        # matched with
        matched_fid = None

        # Now loop over all the trackers and check if the
        # center point of the face is within the box of a
        # tracker
        for fid in face_trackers.keys():
            tracked_position = face_trackers[fid].get_position()
            t_x = int(tracked_position.left())
            t_y = int(tracked_position.top())
            t_w = int(tracked_position.width())
            t_h = int(tracked_position.height())

            # calculate the center point
            t_x_bar = t_x + 0.5 * t_w
            t_y_bar = t_y + 0.5 * t_h

            # check if the center point of the face is within the
            # rectangle of a tracker region. Also, the center point
            # of the tracker region must be within the region
            # detected as a face. If both of these conditions hold
            # we have a match
            if ((t_x <= x_bar <= (t_x + t_w)) and
                    (t_y <= y_bar <= (t_y + t_h)) and
                    (x <= t_x_bar <= (x + w)) and
                    (y <= t_y_bar <= (y + h))):
                matched_fid = fid
                verified_trackers.append(fid)

        if matched_fid is None:
            loc = [x, y, w, h]
            tracker = create_new_tracker(current_face_id, frame, loc)

            face_trackers[current_face_id] = tracker
            verified_trackers.append(current_face_id)
            # Start a new thread that is used to simulate
            # face recognition. This is not yet implemented in this
            # version :)
            # t = threading.Thread(target=do_recognize_person,
            #                      args=(face_names, current_face_id))
            # t.start()
            # Increase the currentFaceID counter
            current_face_id += 1

    # Remove the trackers that are not in the second detection
    for fid in face_trackers.copy().keys():
        if fid not in verified_trackers:
            print("Removing tracker " + str(fid) +
                  " from list of trackers because of not appearing in the second detection")
            del face_trackers[fid]
            

    return current_face_id, face_trackers

In [ ]:
def plot_image_with_boxes(frame, face_trackers, face_name, num_of_digits,
                          frame_number, output_file_name, face_shapes=None, save_image=True):
    face_img = frame.copy()
    for fid in face_trackers.keys():
        tracked_position = face_trackers[fid].get_position()
        t_x = int(tracked_position.left())
        t_y = int(tracked_position.top())
        t_w = int(tracked_position.width())
        t_h = int(tracked_position.height())

        # face = frame[t_y:t_y + t_h, t_x:t_x + t_w]
        face = face_img[t_y:t_y + t_h, t_x:t_x + t_w]
        if face_shapes is not None:
            face = cv2.resize(face, face_shapes, fx=0, fy=0, interpolation=cv2.INTER_CUBIC)

        if fid in face_name.keys():
            box_name = face_name[fid]['name']

        else:
            box_name = 'Detecting...'
            do_recognize_person(face_name, fid, face)
        if save_image:
            if face_name[fid]['detected']:
                directory = '/'.join(output_file_name) + '/labeled/' + face_name[fid]['dir']
                id_person = face_name[fid]['dir'].split('/')[1]
            else:
                directory = '/'.join(output_file_name) + '/unlabeled/' + face_name[fid]['name']
                id_person = face_name[fid]['name']
            # directory = output_file_name + '\\unlabeled\\' + face_name[fid]
            if not os.path.exists(directory):
                os.makedirs(directory)

            num_of_zeros = num_of_digits - len(str(frame_number))
            img_name = directory + '/{source}_{n_video}_{frameNumber}_{id_p}_{x}_{y}_{h}_{w}.jpg'.format(
                source=output_file_name[-2], n_video=output_file_name[-1]
                , frameNumber=num_of_zeros * '0' + str(frame_number)
                , id_p=id_person, x=t_x, y=t_y, h=t_h, w=t_w)

            try:
                cv2.imwrite(img_name, face)
            except cv2.error:
                pass

        cv2.rectangle(frame, (t_x, t_y),
                      (t_x + t_w, t_y + t_h),
                      (0, 0, 255), 2)
        cv2.putText(frame, box_name, (int(t_x + t_w / 2), int(t_y)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (204, 204, 0), 2)

In [ ]:

def face_tracking(path, output_file_name, detection_quality_tsh=7, save_image=True, show_result=True,
                  face_detection_tsh=10, resize_faces_shape=None, desired_time=None):
    # Path to video file
    vidObj = cv2.VideoCapture(path)

    if show_result:
        cv2.namedWindow("Face-Tracking", cv2.WINDOW_AUTOSIZE)  ###make window for image or video

        # Start the window thread for the two windows we are using
        cv2.startWindowThread()

    # Get frame/second of the video
    frames_per_second = int(vidObj.get(cv2.CAP_PROP_FPS))
    print('frames/second:', frames_per_second)

    if desired_time is None:
        frame_threshold = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    else:
        frame_threshold = int(desired_time * 60 * frames_per_second)

    number_of_all_frames_digits = len(str(frame_threshold))

    # Used as counter variable
    frame_counter = 0
    current_face_id = 1

    # Variables holding the correlation trackers and the name per faceid
    face_trackers = {}
    face_names = {}

    while vidObj.isOpened() and frame_counter < frame_threshold:
        # vidObj object calls read
        # function extract frames
        success, image = vidObj.read()

        if success:
            # resize_frame = cv2.resize(image, (1280, 720), fx=0, fy=0, interpolation=cv2.INTER_CUBIC)
            resize_frame = image
            face_trackers = check_tracker_quality(face_trackers, resize_frame,
                                                  quality_thresh_hold=detection_quality_tsh)

            # Every 10 frames, we will have to determine which faces#### is that 10 frames or face_detection_tsh times
            # are present in the frame
            if (frame_counter % face_detection_tsh) == 0:
                all_faces = RetinaFace.detect_faces(resize_frame)
                if type(all_faces) == dict:  ####what is differente in type of faces
                    current_face_id, face_trackers = check_existence_of_new_faces(all_faces.values(), face_trackers,
                                                                                  face_names,
                                                                                  resize_frame, current_face_id)

            frame_counter += 1
            plot_image_with_boxes(resize_frame, face_trackers, face_names, number_of_all_frames_digits,
                                  frame_counter, output_file_name, face_shapes=resize_faces_shape, save_image=save_image)
            if show_result:
                cv2.imshow('Face-Tracking', resize_frame)
                # Press Q on keyboard to  exit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break

        elif (not success) or (frame_counter == frame_threshold):
            vidObj.release()

    if show_result:
        # Destroy any OpenCV windows and exit the application
        cv2.destroyAllWindows()

In [ ]:
def loadFromYT(link, name, output):
  # link = 'https://www.youtube.com/watch?v=o8tXSiKW-h8'
  yt = YouTube(link)
  stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
  stream.download(filename=name, output_path=output)

# Download The Face Dataset

In [ ]:
# import shutil

# shutil.rmtree('frames')

In [ ]:

# !apt install subversion
!npx degit https://github.com/arashHarirpoosh/FaceRecognition/FaceDataBase FaceDataBase --force


npx: installed 1 in 1.743s
> cloned arashHarirpoosh/FaceRecognition#HEAD to FaceDataBase


#Main

In [ ]:
if __name__ == '__main__':
    print('dlib version:', dlib.__version__)

    model = DeepFace.build_model('ArcFace')

    base_addr = 'Videos'

    # num_of_sources = len(os.listdir(base_addr))
    # source_digits = len(str(num_of_sources))
    source_digits = 2
    file_digits = 3
    # selected_channels = ['1', '2']
    selected_channels = ['1']
    selected_videos = [4, 5]
    # 1:CNN, 2:BBC
    videos = {
        '1':
              {
                  '1': 'https://www.youtube.com/watch?v=CmCp6Z54GYU',
                  # '2':'https://youtu.be/pJlnxbO5N2g',
                  # '3':'https://youtu.be/e0sGuSaGvV0',
                  # '4': 'https://youtu.be/ZO_UqxwasO4',
                  # '5':'https://youtu.be/fx2W1Jh4kA0',
                  # '6': 'https://www.youtube.com/watch?v=6y877A9IA4U',
                  # '7':'https://www.youtube.com/watch?v=Rli9ZRvemh0',
                  # '8':'https://www.youtube.com/watch?v=cQAH7d-k068',
                  # '9':'',


              }
        # '2':
        #       {
        #           '1':'https://www.youtube.com/watch?v=sXQLYOHihUg',
                  
        #       }
    }
  
    for channels, videos in videos.items():
        
        root = 'Videos/{c}'.format(c=channels)
        for file_number, link in videos.items():
          file_name = file_number+'.mp4'
          loadFromYT(link=link, name=file_name, output=root)
          root_list = root.split('/')
          root_list[-2] = 'drive/MyDrive/Frames'
          # root_list[-2] = 'frames'
          source_number = root_list[-1]
          root_list[-1] = (source_digits - len(source_number)) * '0' + source_number
          root_list.append((file_digits - len(file_number)) * '0' + file_number)
          p = f'{root}/{file_name}'
          
          print(p)
          print(root_list)

          start_time = time.time()
          face_tracking(
              path=p,
              output_file_name=root_list,
              detection_quality_tsh=7,
              face_detection_tsh=40,
              # resize_faces_shape=(300, 400),
              resize_faces_shape=None,
              save_image=True,
              show_result=False,
              desired_time=8.2
          )

          elapsed_time = time.time() - start_time
          time_report = f'Finished in {elapsed_time} seconds, or {elapsed_time / 60} minutes'
          print(time_report)
          file = open(r"{folder}/time.txt".format(folder='/'.join(root_list)), "w+")
          file.write(time_report)
          file.close()


dlib version: 19.18.0
arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5
100%|██████████| 137M/137M [00:05<00:00, 25.0MB/s]


Videos/1/1.mp4
['/content/drive/MyDrive/Colab Notebooks/master project/try1/frames', '01', '001']
frames/second: 29
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:04<00:00, 24.9MB/s]


Creating new tracker 1
Creating new tracker 2
Creating new tracker 3
Creating new tracker 4
Creating new tracker 5
Creating new tracker 6
Representations stored in  /content/FaceDataBase / representations_arcface.pkl  file. Please delete this file when you add new identities in your database.
find function lasts  19.59169292449951  seconds
Empty DataFrame
Columns: [identity, ArcFace_euclidean_l2]
Index: []

No Face Detected
There are  11  representations found in  representations_arcface.pkl
find function lasts  2.2389719486236572  seconds
Empty DataFrame
Columns: [identity, ArcFace_euclidean_l2]
Index: []

No Face Detected
There are  11  representations found in  representations_arcface.pkl
find function lasts  2.1647496223449707  seconds
Empty DataFrame
Columns: [identity, ArcFace_euclidean_l2]
Index: []

No Face Detected
There are  11  representations found in  representations_arcface.pkl
find function lasts  1.8607795238494873  seconds
Empty DataFrame
Columns: [identity, ArcFace_eu